In [21]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge,Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize


from features import *
from helper import *


In [2]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000


In [3]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [4]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [5]:
x_train_speech = []

counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_speech.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech = np.array(x_train_speech)
x_train_speech.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 100, 34)

In [6]:
def lstm_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(100, 34)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [7]:
model = lstm_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1120256   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 2,041,348
Trainable params: 2,041,348
Non-trainable params: 0
_________________________________________________________________


In [6]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [9]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=60, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/60
3948/3948 [==============================] - 17s 4ms/step - loss: 1.3657 - acc: 0.3544 - val_loss: 1.3529 - val_acc: 0.3826
Epoch 2/60
3948/3948 [==============================] - 16s 4ms/step - loss: 1.3540 - acc: 0.3599 - val_loss: 1.3603 - val_acc: 0.3846
Epoch 3/60
3948/3948 [==============================] - 16s 4ms/step - loss: 1.3543 - acc: 0.3592 - val_loss: 1.3518 - val_acc: 0.3796
Epoch 4/60
3948/3948 [==============================] - 16s 4ms/step - loss: 1.3509 - acc: 0.3622 - val_loss: 1.3451 - val_acc: 0.3897
Epoch 5/60
3948/3948 [==============================] - 16s 4ms/step - loss: 1.3462 - acc: 0.3690 - val_loss: 1.3748 - val_acc: 0.3866
Epoch 6/60
3948/3948 [==============================] - 16s 4ms/step - loss: 1.3511 - acc: 0.3668 - val_loss: 1.3332 - val_acc: 0.3947
Epoch 7/60
3948/3948 [==============================] - 16s 4ms/step - loss: 1.3477 - acc: 0.3668 - val_loss: 1.3518 - val_acc: 0.4018
Epoch 8/

In [10]:
def kaldi_model(optimizer='Adam'):
    model = Sequential()
    model.add(TimeDistributed(Dense(450), input_shape=(100, 34)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(TimeDistributed(Dense(450)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(TimeDistributed(Dense(450)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(TimeDistributed(Dense(450)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(TimeDistributed(Dense(450)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(TimeDistributed(Dense(450)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [11]:
model = kaldi_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 100, 450)          15750     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 450)          1800      
_________________________________________________________________
activation_3 (Activation)    (None, 100, 450)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 450)          202950    
_________________________________________________________________
batch_normalization_2 (Batch (None, 100, 450)          1800      
_________________________________________________________________
activation_4 (Activation)    (None, 100, 450)          0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 100, 450)          202950    
__________

In [12]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=40, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/40
3948/3948 [==============================] - 5s 1ms/step - loss: 11.9052 - acc: 0.2394 - val_loss: 13.4916 - val_acc: 0.1630
Epoch 2/40
3948/3948 [==============================] - 4s 986us/step - loss: 12.2723 - acc: 0.2386 - val_loss: 13.4916 - val_acc: 0.1630
Epoch 3/40
3948/3948 [==============================] - 4s 987us/step - loss: 12.2723 - acc: 0.2386 - val_loss: 13.4916 - val_acc: 0.1630
Epoch 4/40
3948/3948 [==============================] - 4s 990us/step - loss: 12.2723 - acc: 0.2386 - val_loss: 13.4916 - val_acc: 0.1630
Epoch 5/40
3948/3948 [==============================] - 4s 987us/step - loss: 12.2723 - acc: 0.2386 - val_loss: 13.4916 - val_acc: 0.1630
Epoch 6/40
3948/3948 [==============================] - 4s 988us/step - loss: 12.2723 - acc: 0.2386 - val_loss: 13.4916 - val_acc: 0.1630
Epoch 7/40
3948/3948 [==============================] - 4s 988us/step - loss: 12.2723 - acc: 0.2386 - val_loss: 13.4916 - val_a

In [13]:
def linear_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(Flatten(input_shape=(100, 34)))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [14]:
model = linear_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 3400)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_11 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
activation_13 (Activation)   (None, 256)               0         
__________

In [15]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 1s 243us/step - loss: 1.7506 - acc: 0.3488 - val_loss: 1.3135 - val_acc: 0.3937
Epoch 2/80
3948/3948 [==============================] - 1s 127us/step - loss: 1.2712 - acc: 0.4040 - val_loss: 1.3011 - val_acc: 0.3887
Epoch 3/80
3948/3948 [==============================] - 0s 127us/step - loss: 1.1822 - acc: 0.4607 - val_loss: 1.4737 - val_acc: 0.3107
Epoch 4/80
3948/3948 [==============================] - 1s 128us/step - loss: 1.1902 - acc: 0.4488 - val_loss: 1.2535 - val_acc: 0.4038
Epoch 5/80
3948/3948 [==============================] - 1s 128us/step - loss: 1.1178 - acc: 0.4876 - val_loss: 1.5948 - val_acc: 0.3715
Epoch 6/80
3948/3948 [==============================] - 0s 126us/step - loss: 1.1082 - acc: 0.4934 - val_loss: 1.4316 - val_acc: 0.3998
Epoch 7/80
3948/3948 [==============================] - 0s 126us/step - loss: 1.0682 - acc: 0.5301 - val_loss: 1.3327 - val_acc: 0.4059
E

In [16]:
def calculate_features_2(frames, freq, options):
    #double the window duration
    window_sec = 0.4
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [17]:
x_train_speech2 = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features_2(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_speech2.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech2 = np.array(x_train_speech2)
x_train_speech2.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 100, 34)

In [18]:
def linear_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(Flatten(input_shape=(100, 34)))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [19]:
model = linear_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 3400)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_15 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_16 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               131328    
_________________________________________________________________
activation_17 (Activation)   (None, 256)               0         
__________

In [20]:
hist = model.fit(x_train_speech2, Y, 
                 batch_size=100, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 1s 233us/step - loss: 1.5244 - acc: 0.3579 - val_loss: 1.3801 - val_acc: 0.4028
Epoch 2/80
3948/3948 [==============================] - 0s 126us/step - loss: 1.2189 - acc: 0.4412 - val_loss: 1.2390 - val_acc: 0.4302
Epoch 3/80
3948/3948 [==============================] - 0s 123us/step - loss: 1.1846 - acc: 0.4673 - val_loss: 1.2340 - val_acc: 0.4150
Epoch 4/80
3948/3948 [==============================] - 0s 123us/step - loss: 1.1283 - acc: 0.4845 - val_loss: 1.3704 - val_acc: 0.4211
Epoch 5/80
3948/3948 [==============================] - 0s 123us/step - loss: 1.0999 - acc: 0.4919 - val_loss: 1.2273 - val_acc: 0.4767
Epoch 6/80
3948/3948 [==============================] - 0s 124us/step - loss: 1.0920 - acc: 0.5043 - val_loss: 1.4274 - val_acc: 0.3816
Epoch 7/80
3948/3948 [==============================] - 0s 122us/step - loss: 1.0660 - acc: 0.5144 - val_loss: 1.3339 - val_acc: 0.4808
E

In [21]:
def linear_model_combined(optimizer='Adadelta'):
    modela = Sequential()
    modela.add(Flatten(input_shape=(100, 34)))
    modela.add(Dense(1024))
    modela.add(Activation('relu'))
    modela.add(Dense(512))
    
    modelb = Sequential()
    modelb.add(Flatten(input_shape=(100, 34)))
    modelb.add(Dense(1024))
    modelb.add(Activation('relu'))
    modelb.add(Dense(512))
    
    model_combined = Sequential()
    model_combined.add(Merge([modela, modelb], mode='concat'))
    model_combined.add(Activation('relu'))
    model_combined.add(Dense(256))
    model_combined.add(Activation('relu'))
    
    model_combined.add(Dense(4))
    model_combined.add(Activation('softmax'))

    model_combined.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model_combined

In [22]:
model = linear_model_combined()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 1024)              0         
_________________________________________________________________
activation_21 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               262400    
_________________________________________________________________
activation_22 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 1028      
_________________________________________________________________
activation_23 (Activation)   (None, 4)                 0         
Total params: 8,278,276
Trainable params: 8,278,276
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [23]:
hist = model.fit([x_train_speech, x_train_speech2], Y, 
                 batch_size=100, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 1s 340us/step - loss: 2.0201 - acc: 0.3384 - val_loss: 1.4282 - val_acc: 0.3866
Epoch 2/80
3948/3948 [==============================] - 1s 220us/step - loss: 1.2688 - acc: 0.4078 - val_loss: 1.3159 - val_acc: 0.4241
Epoch 3/80
3948/3948 [==============================] - 1s 221us/step - loss: 1.2075 - acc: 0.4491 - val_loss: 1.2337 - val_acc: 0.4190
Epoch 4/80
3948/3948 [==============================] - 1s 221us/step - loss: 1.1629 - acc: 0.4648 - val_loss: 1.3288 - val_acc: 0.4383
Epoch 5/80
3948/3948 [==============================] - 1s 221us/step - loss: 1.1358 - acc: 0.4790 - val_loss: 1.2145 - val_acc: 0.4251
Epoch 6/80
3948/3948 [==============================] - 1s 221us/step - loss: 1.0970 - acc: 0.5000 - val_loss: 1.2968 - val_acc: 0.4140
Epoch 7/80
3948/3948 [==============================] - 1s 221us/step - loss: 1.0950 - acc: 0.5048 - val_loss: 1.4335 - val_acc: 0.3917
E

In [11]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent, _time_distributed_dense
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [14]:
def attention_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(100, 34)))
    model.add(AttentionDecoder(128,128))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [15]:
model = attention_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 128)          83456     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 100, 128)          246528    
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               6554112   
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_6 (Activation)    (None, 4)                 0         
Total para

In [16]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=60, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/60
3948/3948 [==============================] - 25s - loss: 1.3642 - acc: 0.3384 - val_loss: 1.3344 - val_acc: 0.3917
Epoch 2/60
3948/3948 [==============================] - 24s - loss: 1.3507 - acc: 0.3495 - val_loss: 1.3477 - val_acc: 0.3704
Epoch 3/60
3948/3948 [==============================] - 24s - loss: 1.3370 - acc: 0.3544 - val_loss: 1.3278 - val_acc: 0.3745
Epoch 4/60
3948/3948 [==============================] - 24s - loss: 1.2908 - acc: 0.3549 - val_loss: 1.2944 - val_acc: 0.4221
Epoch 5/60
3948/3948 [==============================] - 24s - loss: 1.2461 - acc: 0.4015 - val_loss: 1.3509 - val_acc: 0.2298
Epoch 6/60
3948/3948 [==============================] - 24s - loss: 1.2416 - acc: 0.4098 - val_loss: 1.2243 - val_acc: 0.4636
Epoch 7/60
3948/3948 [==============================] - 24s - loss: 1.1967 - acc: 0.4271 - val_loss: 1.2296 - val_acc: 0.4261
Epoch 8/60
3948/3948 [==============================] - 24s - loss: 1.1

In [17]:
def calculate_features_3(frames, freq, options):
    #double the window duration
    window_sec = 0.08
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [18]:
x_train_speech3 = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features_3(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=200)
    x_train_speech3.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech3 = np.array(x_train_speech3)
x_train_speech3.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 200, 34)

In [12]:
def attention_model2(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(200, 34)))
    model.add(AttentionDecoder(128,128))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [13]:
model = attention_model2()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 200, 128)          83456     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 200, 128)          246528    
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               13107712  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total para

In [25]:
hist = model.fit(x_train_speech3, Y, 
                 batch_size=100, nb_epoch=75, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/75
3948/3948 [==============================] - 62s - loss: 1.3628 - acc: 0.3399 - val_loss: 1.3550 - val_acc: 0.3846
Epoch 2/75
3948/3948 [==============================] - 61s - loss: 1.3551 - acc: 0.3612 - val_loss: 1.3565 - val_acc: 0.3856
Epoch 3/75
3948/3948 [==============================] - 61s - loss: 1.3522 - acc: 0.3597 - val_loss: 1.3615 - val_acc: 0.3836
Epoch 4/75
3948/3948 [==============================] - 61s - loss: 1.3517 - acc: 0.3604 - val_loss: 1.3573 - val_acc: 0.3917
Epoch 5/75
3948/3948 [==============================] - 61s - loss: 1.3518 - acc: 0.3582 - val_loss: 1.3626 - val_acc: 0.3725
Epoch 6/75
3948/3948 [==============================] - 60s - loss: 1.3404 - acc: 0.3582 - val_loss: 1.3488 - val_acc: 0.2885
Epoch 7/75
3948/3948 [==============================] - 61s - loss: 1.3124 - acc: 0.4007 - val_loss: 1.4772 - val_acc: 0.2206
Epoch 8/75
3948/3948 [==============================] - 61s - loss: 1.2

In [7]:
counter = 0
for ses_mod in data2:
    if (ses_mod['id'][:5]=="Ses05"):
        break
    counter+=1
counter

3838

In [8]:
xtrain_sp = x_train_speech[:3838]
xtest_sp = x_train_speech[3838:]
ytrain_sp = Y[:3838]
ytest_sp = Y[3838:]

In [16]:
def attention_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(100, 34)))
    model.add(AttentionDecoder(128,128))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

model = attention_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 128)          83456     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 100, 128)          246528    
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               6554112   
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_4 (Activation)    (None, 4)                 0         
Total para

In [20]:
hist = model.fit(xtrain_sp, ytrain_sp, 
                 batch_size=100, nb_epoch=80, verbose=1, shuffle = True, 
                 validation_data=(xtest_sp, ytest_sp))

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3838 samples, validate on 1098 samples
Epoch 1/80
3838/3838 [==============================] - 25s - loss: 1.3625 - acc: 0.3434 - val_loss: 1.3571 - val_acc: 0.3497
Epoch 2/80
3838/3838 [==============================] - 24s - loss: 1.3441 - acc: 0.3473 - val_loss: 1.3971 - val_acc: 0.2514
Epoch 3/80
3838/3838 [==============================] - 24s - loss: 1.3168 - acc: 0.3781 - val_loss: 1.2859 - val_acc: 0.3816
Epoch 4/80
3838/3838 [==============================] - 24s - loss: 1.2488 - acc: 0.4039 - val_loss: 1.3012 - val_acc: 0.3761
Epoch 5/80
3838/3838 [==============================] - 24s - loss: 1.2164 - acc: 0.4299 - val_loss: 1.2414 - val_acc: 0.3698
Epoch 6/80
3838/3838 [==============================] - 24s - loss: 1.1683 - acc: 0.4567 - val_loss: 1.2050 - val_acc: 0.4454
Epoch 7/80
3838/3838 [==============================] - 24s - loss: 1.1574 - acc: 0.4664 - val_loss: 1.2926 - val_acc: 0.4171
Epoch 8/80
3838/3838 [==============================] - 23s - loss: 1.

In [23]:
def attention_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(100, 34)))
    model.add(Bidirectional(AttentionDecoder(128,128)))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

model = attention_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 100, 256)          166912    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 256)          689664    
_________________________________________________________________
flatten_3 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               13107712  
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_6 (Activation)    (None, 4)                 0         
Total para

In [24]:
hist = model.fit(xtrain_sp, ytrain_sp, 
                 batch_size=100, nb_epoch=100, verbose=1, shuffle = True, 
                 validation_data=(xtest_sp, ytest_sp))

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3838 samples, validate on 1098 samples
Epoch 1/100
3838/3838 [==============================] - 52s - loss: 1.3633 - acc: 0.3421 - val_loss: 1.3600 - val_acc: 0.3497
Epoch 2/100
3838/3838 [==============================] - 50s - loss: 1.3530 - acc: 0.3476 - val_loss: 1.3475 - val_acc: 0.3770
Epoch 3/100
3838/3838 [==============================] - 50s - loss: 1.3347 - acc: 0.3502 - val_loss: 1.4572 - val_acc: 0.2796
Epoch 4/100
3838/3838 [==============================] - 50s - loss: 1.3025 - acc: 0.3890 - val_loss: 1.3198 - val_acc: 0.3661
Epoch 5/100
3838/3838 [==============================] - 51s - loss: 1.2464 - acc: 0.4226 - val_loss: 1.2358 - val_acc: 0.4080
Epoch 6/100
3838/3838 [==============================] - 51s - loss: 1.1896 - acc: 0.4471 - val_loss: 1.5733 - val_acc: 0.3224
Epoch 7/100
3838/3838 [==============================] - 50s - loss: 1.1769 - acc: 0.4567 - val_loss: 1.1983 - val_acc: 0.4262
Epoch 8/100
3838/3838 [==============================] - 50s - 